In [2]:
import json
import numpy as np
from torchvision import datasets, transforms

# Load JSON file
def load_network(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Activation functions
def relu(x):
    return max(0, x)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

def softmax(x):
    e_x = np.exp(x - np.max(x))  # Stability trick to prevent overflow
    return e_x / e_x.sum()

ACTIVATIONS = {
    "relu": relu,
    "sigmoid": sigmoid,
    "tanh": tanh,
    "softmax": softmax
}

# Initialize neuron functions dynamically
def create_neuron(weights, bias, activation, next_nodes):
    activation_func = ACTIVATIONS.get(activation, relu)  # Default to ReLU
    
    def neuron(inputs, results):
        # Compute weighted sum + bias
        z = np.dot(weights, np.array([inputs[n] for n in inputs if n in weights])) + bias
        output = activation_func(z)
        results[id] = output  # Store result
        
        # Trigger next neurons
        for next_node in next_nodes:
            if next_node in results:  # Check if the neuron exists
                results[next_node](inputs, results)
        
        return output
    
    return neuron

# Build layers and neurons
def build_network(json_data):
    layers = {}  # Store layer-wise neurons
    
    for layer_name, layer_info in json_data.items():
        layer_neurons = {}
        
        for node in layer_info['nodes']:
            neuron_id = node['id']
            neuron_func = create_neuron(
                {f"v0_{i:02d}": w for i, w in enumerate(node['weights'])},
                node['biases'],
                node['activation'],
                node['next_nodes']
            )
            layer_neurons[neuron_id] = neuron_func
        
        layers[layer_name] = layer_neurons
    
    return layers

# Forward pass
def forward_pass(layers, input_data):
    results = {}  # Store neuron outputs
    
    # Set input neurons manually
    for neuron_id, value in input_data.items():
        results[neuron_id] = value
    
    # Iterate through layers
    for layer in layers.values():
        for neuron_id, neuron_func in layer.items():
            if neuron_id in results:
                results[neuron_id] = neuron_func(input_data, results)  # Pass input_data and results
    
    return results  # Final output

# Load network
data = load_network("node_based_model.json")
network = build_network(data)

# Load MNIST dataset with correct normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])
mnist_test = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Test on MNIST images
for i in range(10):  # Testing on first 10 images
    image, label = mnist_test[i]
    image = image.view(-1).numpy()  # Flatten 28x28 image to a 784 input vector
    input_data = {f"v0_{j:02d}": image[j] for j in range(784)}  # Convert to expected input format
    
    output = forward_pass(network, input_data)
    print(f"Image {i}: Label = {label}, Network Output = {output}")

TypeError: unsupported operand type(s) for *: 'dict' and 'float'